In [1]:
#Este notebook é primordial em relação a curadoria da base
#A principal função dele é separar as variáveis examinadas em colunas, para uma janela de tempo específica
#ao fim, ele monta uma tabela só, com todas as variáveis e diagnóstico, e salvando em um arquivo .csv

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display

Matplotlib created a temporary cache directory at /tmp/matplotlib-5ezdous9 because the default path (/home/grad/si/22/aliciachaves/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
path = "/home_cerberus/disk2/haniel.botelho/uploads/Downloads/physionet.org/files/eicu-crd/2.0/Base/"

df_patient = pd.read_csv(path + 'patient.csv')
df_apacheApsVar = pd.read_csv(path + 'apacheApsVar.csv')
df_infusiondrug = pd.read_csv(path + 'infusionDrug.csv')
df_physicalExam = pd.read_csv(path + 'physicalExam.csv')
df_lab = pd.read_csv(path + 'lab.csv')
df_hospital = pd.read_csv(path + 'hospital.csv')
df_diagnosis = pd.read_csv(path + 'diagnosis.csv')
df_respiratoryCare = pd.read_csv(path + 'respiratoryCare.csv')
df_medication = pd.read_csv(path + 'medication.csv')
df_microlab = pd.read_csv(path + 'microLab.csv')

/tmp/ipykernel_3669394/3523668563.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_infusiondrug = pd.read_csv(path + 'infusionDrug.csv')


/tmp/ipykernel_3669394/3523668563.py:10: DtypeWarning: Columns (4,5,6,12,26,27,28,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_respiratoryCare = pd.read_csv(path + 'respiratoryCare.csv')


/tmp/ipykernel_3669394/3523668563.py:11: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_medication = pd.read_csv(path + 'medication.csv')


In [3]:
#parametrizando tempo
t0 = -360
t1 = 1440
t2 = 4320

tempo = "alsdkf"
stringquery = f"{tempo} > {t1} and {tempo} < {t2}"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

# Organ disfunction

In [4]:
characteristic = df_patient [['patientunitstayid', 'gender', 'age', 'ethnicity']].copy()
#Substitua valores específicos por NaN
characteristic = characteristic.replace(-1, pd.NA)
characteristic.head()

,patientunitstayid,gender,age,ethnicity
0,141168,Female,70,Caucasian
1,141178,Female,52,Caucasian
2,141179,Female,52,Caucasian
3,141194,Male,68,Caucasian
4,141196,Male,71,Caucasian


### SOFA 1

In [5]:
tempo = "physicalexamoffset"
stringquery = f"{tempo} > {t1} and {tempo} < {t2}"

sofa = df_patient[['patientunitstayid']].copy()

gcs_filtro = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('gcs', case=False, na=False)]
gcs_filtro = gcs_filtro.query(stringquery)
# Convertendo a coluna para valores numéricos, substituindo não numéricos por NaN
gcs_filtro['physicalexamvalue'] = pd.to_numeric(gcs_filtro['physicalexamvalue'], errors='coerce')
# Removendo as linhas que contêm NaN na coluna 'physicalexampath'
gcs_filtro = gcs_filtro.dropna(subset=['physicalexamvalue'])

motor = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Motor Score', case=False, na=False)]
motor = motor.query(stringquery)
motor = motor[['patientunitstayid', 'physicalexamtext']].copy()
motor = motor.rename(columns={'physicalexamtext': 'motor'})
motor['motor'] = motor['motor'].astype(int)
#Removendo motor do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Motor Score', case=False)].index)

eyes = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Eyes Score', case=False, na=False)]
eyes = eyes.query(stringquery)
eyes = eyes[['patientunitstayid', 'physicalexamtext']].copy()
eyes = eyes.rename(columns={'physicalexamtext': 'eyes'})
eyes['eyes'] = eyes['eyes'].astype(int)
#Removendo eyes do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Eyes Score', case=False)].index)

verbal = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Verbal Score', case=False, na=False)]
verbal = verbal.query(stringquery)
verbal = verbal[['patientunitstayid', 'physicalexamtext']].copy()
verbal = verbal.rename(columns={'physicalexamtext': 'verbal'})
verbal['verbal'] = verbal['verbal'].astype(int)
#Removendo verbal do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Verbal Score', case=False)].index)

gcs_total = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('gcs', case=False, na=False)]
gcs_total = gcs_total.query(stringquery)
gcs_total = gcs_total[['patientunitstayid', 'physicalexamtext']].copy()
gcs_total = gcs_total.rename(columns={'physicalexamtext': 'gcs_total'})
gcs_total['gcs_total'] = gcs_total['gcs_total'].astype(int)
#Removendo GCS do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('gcs', case=False)].index)

gcs_temp = pd.merge(motor, eyes, on='patientunitstayid', how='outer')
gcs_temp2 = pd.merge(gcs_temp, verbal, on='patientunitstayid', how='outer')
gcs = pd.merge(gcs_temp2, gcs_total, on='patientunitstayid', how='outer')
gcs ['Total'] = gcs ['motor'] + gcs ['eyes'] + gcs ['verbal']

gcs['Total'] = gcs.apply(
    lambda row: row['gcs_total'] if not np.isnan(row['gcs_total']) and row['Total'] != row['gcs_total'] else row['Total'],
    axis=1
)
gcs = gcs.drop(['motor', 'eyes', 'verbal', 'gcs_total'], axis=1)
idx = gcs.groupby('patientunitstayid')['Total'].idxmin()
idx = idx.dropna()
gcs = gcs.loc[idx]

def score(row):
  if((row['Total']<6)):
    return 4
  elif((row['Total']>=6) and (row['Total']<=9)):
    return 3
  elif((row['Total']>=10) and (row['Total']<=12)):
    return 2
  elif((row['Total']>=13) and (row['Total']<=14)):
    return 1
  else:
    return 0

sofa['Nervous'] = gcs.apply(score, axis=1)
sofa


,patientunitstayid,Nervous
0,141168,0.0
1,141178,0.0
2,141179,0.0
3,141194,0.0
4,141196,0.0
...,...,...
200854,3353235,NaN
200855,3353237,NaN
200856,3353251,NaN
200857,3353254,NaN


In [6]:
tempo = "infusionoffset"
stringquery = f"{tempo} > {t1} and {tempo} < {t2}"

dopamine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Dopamine', case=False, na=False)]
dobutamine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Dobutamine', case=False, na=False)]
epinephrine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Epinephrine', case=False, na=False)]
norepinephrine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Norepinephrine', case=False, na=False)]

dopamine = dopamine.query(stringquery)
dobutamine = dobutamine.query(stringquery)
epinephrine = epinephrine.query(stringquery)
norepinephrine = norepinephrine.query(stringquery)

dopamine = dopamine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()
dobutamine = dobutamine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()
epinephrine = epinephrine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()
norepinephrine = norepinephrine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()

dopamine['concentration.dop'] = dopamine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
epinephrine['concentration.epi'] = epinephrine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
dobutamine['concentration.dob'] = dobutamine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
norepinephrine['concentration.nor'] = norepinephrine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)


dopamine.rename(columns={'drugamount' : 'dopamine'}, inplace=True)
dobutamine.rename(columns={'drugamount' : 'dobutamine'}, inplace=True)
epinephrine.rename(columns={'drugamount' : 'epinephrine'}, inplace=True)
norepinephrine.rename(columns={'drugamount' : 'norepinephrine'}, inplace=True)

dopamine = dopamine.groupby('patientunitstayid')['concentration.dop'].max().reset_index()
dobutamine = dobutamine.groupby('patientunitstayid')['concentration.dob'].max().reset_index()
epinephrine = epinephrine.groupby('patientunitstayid')['concentration.epi'].max().reset_index()
norepinephrine = norepinephrine.groupby('patientunitstayid')['concentration.nor'].max().reset_index()

cardio = pd.merge(dopamine, dobutamine, on='patientunitstayid', how='outer')
cardio = pd.merge(cardio, epinephrine, on='patientunitstayid', how='outer')
cardio = pd.merge(cardio, norepinephrine, on='patientunitstayid', how='outer')

In [7]:
tempo = "physicalexamoffset"
stringquery = f"{tempo} > {t1} and {tempo} < {t2}"

systolic = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('systolic', case=False, na=False)]
diastolic = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('diastolic', case=False, na=False)]

systolic = systolic.query(stringquery)
diastolic = diastolic.query(stringquery)

systolic = systolic[['patientunitstayid', 'physicalexamtext']].copy()
diastolic = diastolic[['patientunitstayid', 'physicalexamtext']].copy()

systolic.rename(columns={'physicalexamtext' : 'systolic'}, inplace=True)
diastolic.rename(columns={'physicalexamtext' : 'diastolic'}, inplace=True)

MAP = pd.merge(systolic, diastolic, on='patientunitstayid')

try:
  MAP["diastolic"] = MAP["diastolic"].astype(int)
except ValueError:
  MAP["diastolic"] = pd.to_numeric(MAP["diastolic"], errors='coerce')

try:
  MAP["systolic"] = MAP["systolic"].astype(int)
except ValueError:
  MAP["systolic"] = pd.to_numeric(MAP["systolic"], errors='coerce')

#PAM = (Pressão Sistólica + 2 x Pressão Diastólica) / 3
MAP['MAP'] = (MAP['systolic'] + 2*(MAP['diastolic'])) / 3
MAP = MAP.groupby('patientunitstayid')['MAP'].min().reset_index()

cardio = pd.merge(cardio, MAP, on='patientunitstayid', how='outer')

In [8]:
def score(row):
  if((row['concentration.dop']>15) or (row['concentration.epi']>0.1) or (row['concentration.nor']>0.1)):
    return 4
  elif((row['concentration.dop']>5) or (row['concentration.epi']<=0.1) or (row['concentration.nor']<=0.1)):
    return 3
  elif((row['concentration.dop']<=5) or (row['concentration.dop']>0)):
    return 2
  elif(row['MAP']<70):
    return 1
  else:
    return 0

sofa['Cardiovascular'] = cardio.apply(score, axis=1)

In [9]:
tempo = "labresultoffset"
stringquery = f"{tempo} > {t1} and {tempo} < {t2}"

pao2 = df_lab[df_lab['labname'].str.contains('^paO2$', case=False, na=False, regex=True)]
fio2 = df_lab[df_lab['labname'].str.contains('^FiO2$', case=False, na=False, regex=True)]

pao2 = pao2.query(stringquery)
fio2 = fio2.query(stringquery)

pao2 = pao2[['patientunitstayid', 'labresult']].copy()
fio2 = fio2[['patientunitstayid', 'labresult']].copy()

pao2.rename(columns={'labresult' : 'pao2'}, inplace=True)
fio2.rename(columns={'labresult' : 'fio2'}, inplace=True)

# Substituir 0.0 por NaN em fio2
fio2['fio2'] = fio2['fio2'].replace(0.0, float('nan'))

resp = pd.merge(pao2, fio2, on='patientunitstayid', how='outer')
resp['Total'] = resp['pao2']/resp['fio2']

def score(row):
    if row['Total'] < 100:
        return 4
    elif 100 <= row['Total'] < 200:
        return 3
    elif 200 <= row['Total'] < 300:
        return 2
    elif 300 <= row['Total'] < 400:
        return 1
    else:
        return 0

resp = resp.groupby('patientunitstayid')['Total'].min().reset_index()
sofa['Respiratory'] = resp.apply(score, axis=1)

In [10]:
tempo = "labresultoffset"
stringquery = f"{tempo} > {t1} and {tempo} < {t2}"

platelets = df_lab[df_lab['labname'].str.contains('platelets', case=False, na=False)]
platelets = platelets.query(stringquery)
platelets = platelets[['patientunitstayid', 'labresult']].copy()
platelets.rename(columns={'labresult' : 'platelets'}, inplace=True)
platelets = platelets.groupby('patientunitstayid')['platelets'].min().reset_index()

def score(row):
    if row['platelets'] < 20:
        return 4
    elif 20 <= row['platelets'] < 50:
        return 3
    elif 50 <= row['platelets'] < 100:
        return 2
    elif 100 <= row['platelets'] < 150:
        return 1
    else:
        return 0

sofa['Coagulation'] = platelets.apply(score, axis=1)

In [11]:
tempo = "labresultoffset"
stringquery = f"{tempo} > {t1} and {tempo} < {t2}"

bilirubin = df_lab[df_lab['labname'].str.contains('total bilirubin', case=False, na=False)]
bilirubin = bilirubin.query(stringquery)
bilirubin = bilirubin[['patientunitstayid', 'labresult']].copy()
bilirubin.rename(columns={'labresult' : 'bilirubin'}, inplace=True)
bilirubin = bilirubin.groupby('patientunitstayid')['bilirubin'].max().reset_index()

def score(row):
    if row['bilirubin'] >= 12.0:
        return 4
    elif 12.0 > row['bilirubin'] >= 6.0:
        return 3
    elif 6.0 > row['bilirubin'] >= 2.0:
        return 2
    elif 2.0 > row['bilirubin'] > 1.2:
        return 1
    else:
        return 0

sofa['Liver'] = bilirubin.apply(score, axis=1)

In [12]:
tempo = "labresultoffset"
stringquery = f"{tempo} > {t1} and {tempo} < {t2}"

creatinine = df_lab[df_lab['labname'].str.contains('creatinine', case=False, na=False)]
creatinine = creatinine.query(stringquery)
creatinine = creatinine[['patientunitstayid','labresult']].copy()
creatinine.rename(columns={'labresult' : 'creatinine'}, inplace=True)
creatinine = creatinine.groupby('patientunitstayid')['creatinine'].max().reset_index()

def score(row):
  if row['creatinine'] >= 5.0:
    return 4
  elif 5.0 > row['creatinine'] >= 3.5:
    return 3
  elif 3.0 > row['creatinine'] >= 2.0:
    return 2
  elif 2.0 > row['creatinine'] > 1.2:
    return 1
  else:
    return 0

sofa['Renal'] = creatinine.apply(score, axis=1)

In [13]:
sofa['sofascore'] = sofa['Nervous'] + sofa['Cardiovascular'] + sofa['Respiratory'] + sofa['Coagulation'] + sofa['Liver'] + sofa['Renal']
sofa = sofa[['patientunitstayid', 'sofascore']].copy()
sofa

,patientunitstayid,sofascore
0,141168,11.0
1,141178,4.0
2,141179,7.0
3,141194,4.0
4,141196,7.0
...,...,...
200854,3353235,NaN
200855,3353237,NaN
200856,3353251,NaN
200857,3353254,NaN


### SOFA2

In [14]:
tempo = "physicalexamoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

gcs_filtro = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('gcs', case=False, na=False)]
gcs_filtro = gcs_filtro.query(querypred)
# Convertendo a coluna para valores numéricos, substituindo não numéricos por NaN
gcs_filtro['physicalexamvalue'] = pd.to_numeric(gcs_filtro['physicalexamvalue'], errors='coerce')
# Removendo as linhas que contêm NaN na coluna 'physicalexampath'
gcs_filtro = gcs_filtro.dropna(subset=['physicalexamvalue'])

motor = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Motor Score', case=False, na=False)]
motor = motor.query(querypred)
motor = motor[['patientunitstayid', 'physicalexamtext']].copy()
motor = motor.rename(columns={'physicalexamtext': 'motor'})
motor['motor'] = motor['motor'].astype(int)
#Removendo motor do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Motor Score', case=False)].index)

eyes = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Eyes Score', case=False, na=False)]
eyes = eyes.query(querypred)
eyes = eyes[['patientunitstayid', 'physicalexamtext']].copy()
eyes = eyes.rename(columns={'physicalexamtext': 'eyes'})
eyes['eyes'] = eyes['eyes'].astype(int)
#Removendo eyes do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Eyes Score', case=False)].index)

verbal = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Verbal Score', case=False, na=False)]
verbal = verbal.query(querypred)
verbal = verbal[['patientunitstayid', 'physicalexamtext']].copy()
verbal = verbal.rename(columns={'physicalexamtext': 'verbal'})
verbal['verbal'] = verbal['verbal'].astype(int)
#Removendo verbal do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Verbal Score', case=False)].index)

gcs_total = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('gcs', case=False, na=False)]
gcs_total = gcs_total.query(querypred)
gcs_total = gcs_total[['patientunitstayid', 'physicalexamtext']].copy()
gcs_total = gcs_total.rename(columns={'physicalexamtext': 'gcs_total'})
gcs_total['gcs_total'] = gcs_total['gcs_total'].astype(int)
#Removendo GCS do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('gcs', case=False)].index)

gcs_temp = pd.merge(motor, eyes, on='patientunitstayid', how='outer')
gcs_temp2 = pd.merge(gcs_temp, verbal, on='patientunitstayid', how='outer')
gcs = pd.merge(gcs_temp2, gcs_total, on='patientunitstayid', how='outer')
gcs ['Total'] = gcs ['motor'] + gcs ['eyes'] + gcs ['verbal']

gcs['Total'] = gcs.apply(
    lambda row: row['gcs_total'] if not np.isnan(row['gcs_total']) and row['Total'] != row['gcs_total'] else row['Total'],
    axis=1
)
gcs = gcs.drop(['motor', 'eyes', 'verbal', 'gcs_total'], axis=1)
idx = gcs.groupby('patientunitstayid')['Total'].idxmin()
idx = idx.dropna()
gcs = gcs.loc[idx]

def score(row):
  if((row['Total']<6)):
    return 4
  elif((row['Total']>=6) and (row['Total']<=9)):
    return 3
  elif((row['Total']>=10) and (row['Total']<=12)):
    return 2
  elif((row['Total']>=13) and (row['Total']<=14)):
    return 1
  else:
    return 0

sofa2 = df_patient[['patientunitstayid']].copy()
sofa2['Nervous'] = gcs.apply(score, axis=1)
sofa2

,patientunitstayid,Nervous
0,141168,0.0
1,141178,NaN
2,141179,NaN
3,141194,NaN
4,141196,NaN
...,...,...
200854,3353235,1.0
200855,3353237,2.0
200856,3353251,1.0
200857,3353254,0.0


In [15]:
tempo = "infusionoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

dopamine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Dopamine', case=False, na=False)]
dobutamine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Dobutamine', case=False, na=False)]
epinephrine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Epinephrine', case=False, na=False)]
norepinephrine = df_infusiondrug[df_infusiondrug['drugname'].str.contains('Norepinephrine', case=False, na=False)]

dopamine = dopamine.query(querypred)
dobutamine = dobutamine.query(querypred)
epinephrine = epinephrine.query(querypred)
norepinephrine = norepinephrine.query(querypred)

dopamine = dopamine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()
dobutamine = dobutamine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()
epinephrine = epinephrine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()
norepinephrine = norepinephrine[['patientunitstayid', 'drugamount', 'volumeoffluid']].copy()

dopamine['concentration.dop'] = dopamine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
epinephrine['concentration.epi'] = epinephrine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
dobutamine['concentration.dob'] = dobutamine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)
norepinephrine['concentration.nor'] = norepinephrine.apply(lambda row: row['drugamount'] / row['volumeoffluid'] if row['volumeoffluid'] else None, axis=1)


dopamine.rename(columns={'drugamount' : 'dopamine'}, inplace=True)
dobutamine.rename(columns={'drugamount' : 'dobutamine'}, inplace=True)
epinephrine.rename(columns={'drugamount' : 'epinephrine'}, inplace=True)
norepinephrine.rename(columns={'drugamount' : 'norepinephrine'}, inplace=True)

dopamine = dopamine.groupby('patientunitstayid')['concentration.dop'].max().reset_index()
dobutamine = dobutamine.groupby('patientunitstayid')['concentration.dob'].max().reset_index()
epinephrine = epinephrine.groupby('patientunitstayid')['concentration.epi'].max().reset_index()
norepinephrine = norepinephrine.groupby('patientunitstayid')['concentration.nor'].max().reset_index()

cardio = pd.merge(dopamine, dobutamine, on='patientunitstayid', how='outer')
cardio = pd.merge(cardio, epinephrine, on='patientunitstayid', how='outer')
cardio = pd.merge(cardio, norepinephrine, on='patientunitstayid', how='outer')

In [16]:
tempo = "physicalexamoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

systolic = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('systolic', case=False, na=False)]
diastolic = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('diastolic', case=False, na=False)]

systolic = systolic.query(querypred)
diastolic = diastolic.query(querypred)

systolic = systolic[['patientunitstayid', 'physicalexamtext']].copy()
diastolic = diastolic[['patientunitstayid', 'physicalexamtext']].copy()

systolic.rename(columns={'physicalexamtext' : 'systolic'}, inplace=True)
diastolic.rename(columns={'physicalexamtext' : 'diastolic'}, inplace=True)

MAP = pd.merge(systolic, diastolic, on='patientunitstayid')

try:
  MAP["diastolic"] = MAP["diastolic"].astype(int)
except ValueError:
  MAP["diastolic"] = pd.to_numeric(MAP["diastolic"], errors='coerce')

try:
  MAP["systolic"] = MAP["systolic"].astype(int)
except ValueError:
  MAP["systolic"] = pd.to_numeric(MAP["systolic"], errors='coerce')

#PAM = (Pressão Sistólica + 2 x Pressão Diastólica) / 3
MAP['MAP'] = (MAP['systolic'] + 2*(MAP['diastolic'])) / 3
MAP = MAP.groupby('patientunitstayid')['MAP'].min().reset_index()

cardio = pd.merge(cardio, MAP, on='patientunitstayid', how='outer')

In [17]:
def score(row):
    if((row['concentration.dop']>15) or (row['concentration.epi']>0.1) or (row['concentration.nor']>0.1)):
        return 4
    elif((row['concentration.dop']>5) or (row['concentration.epi']<=0.1) or (row['concentration.nor']<=0.1)):
        return 3
    elif((row['concentration.dop']<=5) or (row['concentration.dop']>0)):
        return 2
    elif(row['MAP']<70):
        return 1
    else:
        return 0

sofa2['Cardiovascular'] = cardio.apply(score, axis=1)

In [18]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

pao2 = df_lab[df_lab['labname'].str.contains('^paO2$', case=False, na=False, regex=True)]
fio2 = df_lab[df_lab['labname'].str.contains('^FiO2$', case=False, na=False, regex=True)]

pao2 = pao2.query(querypred)
fio2 = fio2.query(querypred)

pao2 = pao2[['patientunitstayid', 'labresult']].copy()
fio2 = fio2[['patientunitstayid', 'labresult']].copy()

pao2.rename(columns={'labresult' : 'pao2'}, inplace=True)
fio2.rename(columns={'labresult' : 'fio2'}, inplace=True)

# Substituir 0.0 por NaN em fio2
fio2['fio2'] = fio2['fio2'].replace(0.0, float('nan'))

resp = pd.merge(pao2, fio2, on='patientunitstayid', how='outer')
resp['Total'] = resp['pao2']/resp['fio2']

def score(row):
    if row['Total'] < 100:
        return 4
    elif 100 <= row['Total'] < 200:
        return 3
    elif 200 <= row['Total'] < 300:
        return 2
    elif 300 <= row['Total'] < 400:
        return 1
    else:
        return 0

resp = resp.groupby('patientunitstayid')['Total'].min().reset_index()
sofa2['Respiratory'] = resp.apply(score, axis=1)

In [19]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

platelets = df_lab[df_lab['labname'].str.contains('platelets', case=False, na=False)]
platelets = platelets.query(querypred)
platelets = platelets[['patientunitstayid', 'labresult']].copy()
platelets.rename(columns={'labresult' : 'platelets'}, inplace=True)
platelets = platelets.groupby('patientunitstayid')['platelets'].min().reset_index()

def score(row):
    if row['platelets'] < 20:
        return 4
    elif 20 <= row['platelets'] < 50:
        return 3
    elif 50 <= row['platelets'] < 100:
        return 2
    elif 100 <= row['platelets'] < 150:
        return 1
    else:
        return 0

sofa2['Coagulation'] = platelets.apply(score, axis=1)

In [20]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

bilirubin = df_lab[df_lab['labname'].str.contains('total bilirubin', case=False, na=False)]
bilirubin = bilirubin.query(querypred)
bilirubin = bilirubin[['patientunitstayid', 'labresult']].copy()
bilirubin.rename(columns={'labresult' : 'bilirubin'}, inplace=True)
bilirubin = bilirubin.groupby('patientunitstayid')['bilirubin'].max().reset_index()

def score(row):
    if row['bilirubin'] >= 12.0:
        return 4
    elif 12.0 > row['bilirubin'] >= 6.0:
        return 3
    elif 6.0 > row['bilirubin'] >= 2.0:
        return 2
    elif 2.0 > row['bilirubin'] > 1.2:
        return 1
    else:
        return 0

sofa2['Liver'] = bilirubin.apply(score, axis=1)

In [21]:
creatinine = df_lab[df_lab['labname'].str.contains('creatinine', case=False, na=False)]
creatinine = creatinine.query('labresultoffset <= 360')
creatinine = creatinine[['patientunitstayid','labresult']].copy()
creatinine.rename(columns={'labresult' : 'creatinine'}, inplace=True)
creatinine = creatinine.groupby('patientunitstayid')['creatinine'].max().reset_index()

def score(row):
  if row['creatinine'] >= 5.0:
    return 4
  elif 5.0 > row['creatinine'] >= 3.5:
    return 3
  elif 3.0 > row['creatinine'] >= 2.0:
    return 2
  elif 2.0 > row['creatinine'] > 1.2:
    return 1
  else:
    return 0

sofa2['Renal'] = creatinine.apply(score, axis=1)

In [22]:
sofa2['sofa2score'] = sofa2['Nervous'] + sofa2['Cardiovascular'] + sofa2['Respiratory'] + sofa2['Coagulation'] + sofa2['Liver'] + sofa2['Renal']
sofa2 = sofa2[['patientunitstayid', 'sofa2score']].copy()
sofa2

,patientunitstayid,sofa2score
0,141168,3.0
1,141178,NaN
2,141179,NaN
3,141194,NaN
4,141196,NaN
...,...,...
200854,3353235,NaN
200855,3353237,NaN
200856,3353251,NaN
200857,3353254,NaN


# Inflammation

In [23]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

bands = df_lab[df_lab['labname'].str.contains('-bands', case=False, na=False)]
CRP = df_lab[df_lab['labname'].str.contains('CRP', case=False, na=False)]
ESR = df_lab[df_lab['labname'].str.contains('ESR', case=False, na=False)]
lactate = df_lab[df_lab['labname'].str.contains('lactate', case=False, na=False)]
temperature = df_lab[df_lab['labname'].str.contains('Temp', case=False, na=False)]
WBC = df_lab[df_lab['labname'].str.contains('WBC x 1000', case=False, na=False)]
lymphs = df_lab[df_lab['labname'].str.contains('-lymphs', case=False, na=False)]

bands = bands.query(querypred)
CRP = CRP.query(querypred)
ESR = ESR.query(querypred)
temperature = temperature.query(querypred)
lactate = lactate.query(querypred)
WBC = WBC.query(querypred)
lymphs = lymphs.query(querypred)

bands = bands[['patientunitstayid', 'labresult']].copy()
CRP = CRP[['patientunitstayid', 'labresult']].copy()
ESR = ESR[['patientunitstayid', 'labresult']].copy()
temperature = temperature[['patientunitstayid', 'labresulttext']].copy()
lactate = lactate[['patientunitstayid', 'labresult']].copy()
WBC = WBC[['patientunitstayid', 'labresult']].copy()
lymphs = lymphs[['patientunitstayid', 'labresult']].copy()

# Transformando as temperaturas em celsius
temperature['temp'] = temperature['labresulttext'].astype(float)
temperature['temp'] = temperature['temp'].map(lambda x: x if x < 60 else (x - 32) * 5/9)

# Pior valor da temperatura
temperature['distancia'] = np.abs(temperature['temp'] - 36.85)
temperature = temperature.loc[temperature.groupby('patientunitstayid')['distancia'].idxmax()]
temperature = temperature.reset_index(drop=True)
temperature = temperature.drop(['distancia', 'labresulttext'], axis=1)

# Calcular o valor mais crítico para WBC
WBC = WBC.dropna(subset=['labresult'])
WBC['distancia'] = np.abs(WBC['labresult'] - 7500)
WBC = WBC.loc[WBC.groupby('patientunitstayid')['distancia'].idxmax()]
WBC = WBC.reset_index(drop=True)
WBC = WBC.drop(['distancia'], axis=1)

# Calcular o valor mais crítico para linfócitos
lymphs = lymphs.dropna(subset=['labresult'])
lymphs['distancia'] = np.abs(lymphs['labresult'] - 2000)
lymphs = lymphs.loc[lymphs.groupby('patientunitstayid')['distancia'].idxmax()]
lymphs = lymphs.reset_index(drop=True)
lymphs = lymphs.drop(['distancia'], axis=1)

bands = bands.groupby('patientunitstayid')['labresult'].max().reset_index()
CRP = CRP.groupby('patientunitstayid')['labresult'].max().reset_index()
ESR = ESR.groupby('patientunitstayid')['labresult'].max().reset_index()
lactate= lactate.groupby('patientunitstayid')['labresult'].max().reset_index()

bands = bands.rename(columns={'labresult': 'bands'})
CRP = CRP.rename(columns={'labresult': 'CRP'})
ESR = ESR.rename(columns={'labresult': 'ESR'})
lactate = lactate.rename(columns={'labresult': 'lactate'})
WBC = WBC.rename(columns={'labresult': 'WBC'})
lymphs = lymphs.rename(columns={'labresult': 'lymphs'})

inflammation = pd.merge(bands, CRP, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, ESR, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, temperature, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, lactate, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, WBC, on='patientunitstayid', how='outer')
inflammation = pd.merge(inflammation, lymphs, on='patientunitstayid', how='outer')

inflammation

,patientunitstayid,bands,CRP,ESR,temp,lactate,WBC,lymphs
0,141168,NaN,NaN,NaN,NaN,NaN,9.8,6.0
1,141178,NaN,NaN,NaN,NaN,NaN,7.6,45.0
2,141194,NaN,NaN,NaN,NaN,1.9,14.1,1.0
3,141196,NaN,NaN,NaN,NaN,0.8,26.2,3.0
4,141197,NaN,NaN,NaN,NaN,NaN,29.5,3.0
...,...,...,...,...,...,...,...,...
180605,3353235,NaN,NaN,NaN,NaN,NaN,5.4,13.0
180606,3353237,NaN,NaN,NaN,NaN,NaN,9.5,NaN
180607,3353251,NaN,NaN,NaN,NaN,9.8,20.4,8.0
180608,3353254,NaN,NaN,NaN,NaN,1.3,13.5,7.0


# Pulmonary

In [24]:
tempo = "physicalexamoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

resp = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('Resp Current', case=False, na=False)]
resp = resp.query(querypred)
resp = resp[['patientunitstayid', 'physicalexamtext']].copy()
resp['physicalexamtext'] = resp['physicalexamtext'].astype(float)

resp['distancia'] = np.abs(resp['physicalexamtext'] - 16)
resp = resp.loc[resp.groupby('patientunitstayid')['distancia'].idxmax()]
resp = resp.reset_index(drop=True)
resp = resp.drop(['distancia'], axis=1)

resp = resp.groupby('patientunitstayid')['physicalexamtext'].max().reset_index()
resp = resp.rename(columns={'physicalexamtext': 'resp'})

In [25]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

spo2 = df_lab[df_lab['labname'].str.contains('O2 Sat', case=False, na=False)]
fio2 = df_lab[df_lab['labname'].str.contains('FiO2', case=False, na=False)]

spo2 = spo2.query(querypred)
fio2 = fio2.query(querypred)

spo2 = spo2[['patientunitstayid', 'labresult']].copy()
fio2 = fio2[['patientunitstayid', 'labresult']].copy()

spo2 = spo2.rename(columns={'labresult': 'spo2'})
fio2 = fio2.rename(columns={'labresult': 'fio2'})

# Substituir 0.0 por NaN em fio2
fio2['fio2'] = fio2['fio2'].replace(0.0, float('nan'))

spo2_fio2 = pd.merge(spo2, fio2, on='patientunitstayid')
spo2_fio2['spo2/fio2'] = spo2_fio2['spo2'] / spo2_fio2['fio2']
spo2_fio2 = spo2_fio2.drop(['spo2', 'fio2'], axis=1)
spo2_fio2 = spo2_fio2.groupby('patientunitstayid')['spo2/fio2'].max().reset_index()

In [26]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

pao2 = df_lab[df_lab['labname'].str.contains('paO2', case=False, na=False)]
pao2 = pao2.query(querypred)
pao2 = pao2[['patientunitstayid', 'labresult']].copy()
pao2 = pao2.rename(columns={'labresult': 'pao2'})

# Substituir 0.0 por NaN em fio2
fio2['fio2'] = fio2['fio2'].replace(0.0, float('nan'))

pao2_fio2 = pd.merge(pao2, fio2, on='patientunitstayid')

pao2_fio2['pao2/fio2'] = pao2_fio2['pao2'] / pao2_fio2['fio2']
pao2_fio2 = pao2_fio2.drop(['pao2', 'fio2'], axis=1)
pao2_fio2 = pao2_fio2.groupby('patientunitstayid')['pao2/fio2'].min().reset_index()

In [27]:
pulmonary = pd.merge(resp, spo2_fio2, on='patientunitstayid', how='outer')
pulmonary = pd.merge(pulmonary, pao2_fio2, on='patientunitstayid', how='outer')
pulmonary

,patientunitstayid,resp,spo2/fio2,pao2/fio2
0,141194,25.0,NaN,NaN
1,141197,24.0,NaN,NaN
2,141203,NaN,NaN,0.510000
3,141227,NaN,NaN,0.640000
4,141233,NaN,2.475,1.020000
...,...,...,...,...
142038,3353226,14.0,100.000,2.185714
142039,3353237,29.0,NaN,NaN
142040,3353251,24.0,100.000,0.800000
142041,3353254,24.0,NaN,NaN


# Cardiovascular

In [28]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

bicarbonate = df_lab[df_lab['labname'].str.contains('bicarbonate', case=False, na=False)]
bicarbonate = bicarbonate.query(querypred)
bicarbonate = bicarbonate[['patientunitstayid', 'labresult']].copy()
bicarbonate.rename(columns={'labresult': 'bicarbonate'}, inplace=True)
bicarbonate = bicarbonate.groupby('patientunitstayid')['bicarbonate'].min().reset_index()

In [29]:
tempo = "physicalexamoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

heartRate = df_physicalExam[df_physicalExam['physicalexamvalue'].str.contains('HR Current', case=False, na=False)]
heartRate = heartRate.query(querypred)
heartRate = heartRate[['patientunitstayid', 'physicalexamtext']].copy()

heartRate['physicalexamtext'] = heartRate['physicalexamtext'].astype(float)
heartRate['distancia'] = np.abs(heartRate['physicalexamtext'] - 75)
heartRate = heartRate.loc[heartRate.groupby('patientunitstayid')['distancia'].idxmax()]
heartRate = heartRate.reset_index(drop=True)
heartRate = heartRate.drop(['distancia'], axis=1)

heartRate.rename(columns={'physicalexamtext': 'heartRate'}, inplace=True)

In [30]:
tempo = "physicalexamoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

systolic = df_physicalExam[df_physicalExam['physicalexamvalue'].str.contains(r'systolic.*Current|Current.*systolic', case=False, na=False)]
systolic = systolic.query(querypred)
systolic = systolic[['patientunitstayid', 'physicalexamtext']].copy()

systolic['physicalexamtext'] = systolic['physicalexamtext'].map(lambda x: -1 if not x.isnumeric() else float(x))

systolic['distancia'] = np.abs(systolic['physicalexamtext'] - 115)
systolic = systolic.loc[systolic.groupby('patientunitstayid')['distancia'].idxmax()]
systolic = systolic.drop_duplicates(subset='patientunitstayid')
systolic = systolic.reset_index(drop=True)
systolic = systolic.drop(['distancia'], axis=1)

systolic.rename(columns={'physicalexamtext': 'systolic'}, inplace=True)

In [31]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

troponin = df_lab[df_lab['labname'].str.contains('troponin', case=False, na=False)]
troponin = troponin.query(querypred)
troponin = troponin[['patientunitstayid', 'labresult']].copy()
troponin.rename(columns={'labresult': 'troponin'}, inplace=True)
troponin = troponin.groupby('patientunitstayid')['troponin'].max().reset_index()

In [32]:
tempo = "physicalexamoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

diastolic = df_physicalExam[df_physicalExam['physicalexamvalue'].str.contains(r'diastolic.*Current|Current.*diastolic', case=False, na=False)]
diastolic = diastolic.query(querypred)
diastolic = diastolic[['patientunitstayid', 'physicalexamtext']].copy()
diastolic['physicalexamtext'] = diastolic['physicalexamtext'].map(lambda x: -1 if not x.isnumeric() else float(x))

diastolic['distancia'] = np.abs(diastolic['physicalexamtext'] - 62.5)
diastolic = diastolic.loc[diastolic.groupby('patientunitstayid')['distancia'].idxmax()]
diastolic = diastolic.drop_duplicates(subset='patientunitstayid')
diastolic = diastolic.reset_index(drop=True)
diastolic = diastolic.drop(['distancia'], axis=1)

diastolic.rename(columns={'physicalexamtext': 'diastolic'}, inplace=True)

In [33]:
tempo = "physicalexamoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

systolic_map = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('systolic', case=False, na=False)]
diastolic_map = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('diastolic', case=False, na=False)]

systolic_map = systolic_map.query(querypred)
diastolic_map = diastolic_map.query(querypred)

systolic_map = systolic_map[['patientunitstayid', 'physicalexamtext']].copy()
diastolic_map = diastolic_map[['patientunitstayid', 'physicalexamtext']].copy()

systolic_map.rename(columns={'physicalexamtext' : 'systolic'}, inplace=True)
diastolic_map.rename(columns={'physicalexamtext' : 'diastolic'}, inplace=True)

MAP = pd.merge(systolic_map, diastolic_map, on='patientunitstayid')

try:
  MAP["diastolic"] = MAP["diastolic"].astype(int)
except ValueError:
  MAP["diastolic"] = pd.to_numeric(MAP["diastolic"], errors='coerce')

try:
  MAP["systolic"] = MAP["systolic"].astype(int)
except ValueError:
  MAP["systolic"] = pd.to_numeric(MAP["systolic"], errors='coerce')

MAP['MAP'] = (MAP['systolic'] + 2*(MAP['diastolic'])) / 3
MAP = MAP.groupby('patientunitstayid')['MAP'].min().reset_index()

In [34]:
cardiovascular = pd.merge(bicarbonate, heartRate, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, troponin, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, systolic, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, diastolic, on='patientunitstayid', how='outer')
cardiovascular = pd.merge(cardiovascular, MAP, on='patientunitstayid', how='outer')
cardiovascular

,patientunitstayid,bicarbonate,heartRate,troponin,systolic,diastolic,MAP
0,141168,22.0,NaN,NaN,NaN,NaN,NaN
1,141178,25.0,NaN,NaN,NaN,NaN,NaN
2,141179,22.0,NaN,NaN,NaN,NaN,NaN
3,141194,15.0,112.0,NaN,84.0,34.0,50.666667
4,141196,30.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
189659,3353235,24.0,NaN,0.04,161.0,98.0,119.000000
189660,3353237,27.0,91.0,0.19,144.0,80.0,78.333333
189661,3353251,19.0,90.0,9.38,107.0,54.0,71.666667
189662,3353254,20.0,76.0,NaN,80.0,50.0,60.000000


# Renal

In [35]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

BUN = df_lab[df_lab['labname'].str.contains('BUN', case=False, na=False)]
BUN = BUN.query(querypred)
BUN = BUN[['patientunitstayid', 'labresult']].copy()
BUN.rename(columns={'labresult' : 'bloodUreaNitrogen'}, inplace=True)
BUN = BUN.groupby('patientunitstayid')['bloodUreaNitrogen'].max().reset_index()
renal = BUN

renal

,patientunitstayid,bloodUreaNitrogen
0,141168,27.0
1,141178,11.0
2,141179,21.0
3,141194,31.0
4,141196,16.0
...,...,...
181279,3353235,16.0
181280,3353237,32.0
181281,3353251,31.0
181282,3353254,45.0


# Hepatic

In [36]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

# Selecionar indicadores hepáticos no dataset 'lab'
alanine = df_lab[df_lab['labname'].str.contains('ALT', case=False, na=False)]
aspartate = df_lab[df_lab['labname'].str.contains('AST', case=False, na=False)]
bilirubin = df_lab[df_lab['labname'].str.contains('total bilirubin', case=False, na=False)]

# Filtrar para o intervalo das primeiras 24 horas do paciente na UTI
alanine = alanine.query(querypred)
aspartate = aspartate.query(querypred)
bilirubin = bilirubin.query(querypred)

# Fazer uma cópia com as colunas importantes para o modelo
alanine = alanine[['patientunitstayid', 'labresult']].copy()
aspartate = aspartate[['patientunitstayid', 'labresult']].copy()
bilirubin = bilirubin[['patientunitstayid', 'labresult']].copy()

# Renomear colunas para facilitar visualização
alanine.rename(columns={'labresult' : 'alanine'}, inplace=True)
aspartate.rename(columns={'labresult' : 'aspartate'}, inplace=True)
bilirubin.rename(columns={'labresult' : 'bilirubin'}, inplace=True)

# Filtrar os valores mais críticos de cada paciente
alanine = alanine.groupby('patientunitstayid')['alanine'].max().reset_index()
aspartate = aspartate.groupby('patientunitstayid')['aspartate'].max().reset_index()
bilirubin = bilirubin.groupby('patientunitstayid')['bilirubin'].max().reset_index()

# Mesclar os datasets
hepatic = pd.merge(alanine, aspartate, on='patientunitstayid', how='outer')
hepatic = pd.merge(hepatic, bilirubin, on='patientunitstayid', how='outer')

hepatic

,patientunitstayid,alanine,aspartate,bilirubin
0,141168,358.0,878.0,4.1
1,141178,52.0,40.0,0.4
2,141194,20.0,24.0,0.4
3,141196,18.0,15.0,0.3
4,141197,19.0,17.0,0.3
...,...,...,...,...
107722,3353216,9.0,20.0,0.2
107723,3353226,36.0,31.0,0.4
107724,3353237,12.0,18.0,0.7
107725,3353251,272.0,264.0,0.5


# Hematologic

In [37]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

hemoglobin = df_lab[df_lab['labname'].str.contains('Hgb', case=False, na=False)]
hemoglobin = hemoglobin.query(querypred)
hemoglobin = hemoglobin[['patientunitstayid', 'labresult']].copy()
hemoglobin.rename(columns={'labresult' : 'hemoglobin'}, inplace=True)
hemoglobin = hemoglobin.groupby('patientunitstayid')['hemoglobin'].min().reset_index()

In [38]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

iqr = df_lab[df_lab['labname'].str.contains('INR', case=False, na=False)]
iqr = iqr.query(querypred)
iqr = iqr[['patientunitstayid', 'labresult']].copy()
iqr.rename(columns={'labresult' : 'iqr'}, inplace=True)
iqr = iqr.groupby('patientunitstayid')['iqr'].max().reset_index()

In [39]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

platelets = df_lab[df_lab['labname'].str.contains('platelets', case=False, na=False)]
platelets = platelets.query(querypred)
platelets = platelets[['patientunitstayid', 'labresult']].copy()
platelets.rename(columns={'labresult' : 'platelets'}, inplace=True)
platelets = platelets.groupby('patientunitstayid')['platelets'].min().reset_index()

In [40]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

PTT = df_lab[df_lab['labname'].str.contains('PTT', case=False, na=False)]
PTT = PTT.query(querypred)
PTT = PTT[['patientunitstayid', 'labresult']].copy()
PTT.rename(columns={'labresult' : 'PTT'}, inplace=True)
PTT = PTT.groupby('patientunitstayid')['PTT'].max().reset_index()

In [41]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

fibrinogen = df_lab[df_lab['labname'].str.contains('fibrinogen', case=False, na=False)]
fibrinogen = fibrinogen.query(querypred)
fibrinogen = fibrinogen[['patientunitstayid', 'labresult']].copy()

fibrinogen['distancia'] = np.abs(fibrinogen['labresult'] - 300)
fibrinogen = fibrinogen.dropna(subset=['distancia'])
fibrinogen = fibrinogen.loc[fibrinogen.groupby('patientunitstayid')['distancia'].idxmax()]
fibrinogen = fibrinogen.reset_index(drop=True)
fibrinogen = fibrinogen.drop(['distancia'], axis=1)

fibrinogen.rename(columns={'labresult' : 'fibrinogen'}, inplace=True)

In [42]:
hematologic = pd.merge(hemoglobin, iqr, on='patientunitstayid', how='outer')
hematologic = pd.merge(hematologic, platelets, on='patientunitstayid', how='outer')
hematologic = pd.merge(hematologic, PTT, on='patientunitstayid', how='outer')
hematologic = pd.merge(hematologic, fibrinogen, on='patientunitstayid', how='outer')
hematologic

,patientunitstayid,hemoglobin,iqr,platelets,PTT,fibrinogen
0,141168,13.0,2.50,209.0,29.0,NaN
1,141178,15.5,NaN,273.0,NaN,NaN
2,141194,8.9,NaN,233.0,24.0,NaN
3,141196,10.3,NaN,453.0,NaN,NaN
4,141197,11.5,NaN,589.0,NaN,NaN
...,...,...,...,...,...,...
181934,3353235,16.3,NaN,133.0,NaN,NaN
181935,3353237,13.4,1.27,128.0,114.0,NaN
181936,3353251,10.2,1.14,196.0,24.0,NaN
181937,3353254,7.3,1.03,183.0,26.0,NaN


# Bioquimica

In [43]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

albumin = df_lab[df_lab['labname'].str.contains('albumin', case=False, na=False)]
albumin = albumin.query(querypred)
albumin = albumin[['patientunitstayid', 'labresult']].copy()
albumin.rename(columns={'labresult' : 'albumin'}, inplace=True)
albumin = albumin.groupby('patientunitstayid')['albumin'].min().reset_index()

In [44]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

chloride = df_lab[df_lab['labname'].str.contains('chloride', case=False, na=False)]
chloride = chloride.query(querypred)
chloride = chloride[['patientunitstayid', 'labresult']].copy()

chloride['distancia'] = np.abs(chloride['labresult'] - 102.5)
chloride = chloride.dropna(subset=['distancia'])
chloride = chloride.loc[chloride.groupby('patientunitstayid')['distancia'].idxmax()]
chloride = chloride.reset_index(drop=True)
chloride = chloride.drop(['distancia'], axis=1)

chloride.rename(columns={'labresult' : 'chloride'}, inplace=True)

In [45]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

glucose = df_lab[df_lab['labname'].str.contains('glucose', case=False, na=False)]
glucose = glucose.query(querypred)
glucose = glucose[['patientunitstayid', 'labresult']].copy()

glucose['distancia'] = np.abs(glucose['labresult'] - 125)
glucose = glucose.dropna(subset=['distancia'])
glucose = glucose.loc[glucose.groupby('patientunitstayid')['distancia'].idxmax()]
glucose = glucose.reset_index(drop=True)
glucose = glucose.drop(['distancia'], axis=1)

glucose.rename(columns={'labresult' : 'glucose'}, inplace=True)

In [46]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

sodium = df_lab[df_lab['labname'].str.contains('sodium', case=False, na=False)]
sodium = sodium.query(querypred)
sodium = sodium[['patientunitstayid', 'labresult']].copy()

sodium['distancia'] = np.abs(sodium['labresult'] - 140)
sodium = sodium.dropna(subset=['distancia'])
sodium = sodium.loc[sodium.groupby('patientunitstayid')['distancia'].idxmax()]
sodium = sodium.reset_index(drop=True)
sodium = sodium.drop(['distancia'], axis=1)

sodium.rename(columns={'labresult' : 'sodium'}, inplace=True)

In [47]:
tempo = "physicalexamoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

gcs_filtro = df_physicalExam[df_physicalExam['physicalexampath'].str.contains('gcs', case=False, na=False)]
gcs_filtro = gcs_filtro.query(querypred)
# Convertendo a coluna para valores numéricos, substituindo não numéricos por NaN
gcs_filtro['physicalexamvalue'] = pd.to_numeric(gcs_filtro['physicalexamvalue'], errors='coerce')
# Removendo as linhas que contêm NaN na coluna 'physicalexampath'
gcs_filtro = gcs_filtro.dropna(subset=['physicalexamvalue'])

motor = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Motor Score', case=False, na=False)]
motor = motor.query(querypred)
motor = motor[['patientunitstayid', 'physicalexamtext']].copy()
motor = motor.rename(columns={'physicalexamtext': 'motor'})
motor['motor'] = motor['motor'].astype(int)
#Removendo motor do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Motor Score', case=False)].index)

eyes = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Eyes Score', case=False, na=False)]
eyes = eyes.query(querypred)
eyes = eyes[['patientunitstayid', 'physicalexamtext']].copy()
eyes = eyes.rename(columns={'physicalexamtext': 'eyes'})
eyes['eyes'] = eyes['eyes'].astype(int)
#Removendo eyes do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Eyes Score', case=False)].index)

verbal = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Verbal Score', case=False, na=False)]
verbal = verbal.query(querypred)
verbal = verbal[['patientunitstayid', 'physicalexamtext']].copy()
verbal = verbal.rename(columns={'physicalexamtext': 'verbal'})
verbal['verbal'] = verbal['verbal'].astype(int)
#Removendo verbal do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('Verbal Score', case=False)].index)

gcs_total = gcs_filtro[gcs_filtro['physicalexampath'].str.contains('gcs', case=False, na=False)]
gcs_total = gcs_total.query(querypred)
gcs_total = gcs_total[['patientunitstayid', 'physicalexamtext']].copy()
gcs_total = gcs_total.rename(columns={'physicalexamtext': 'gcs_total'})
gcs_total['gcs_total'] = gcs_total['gcs_total'].astype(int)
#Removendo GCS do df GCS
gcs_filtro = gcs_filtro.drop(gcs_filtro[gcs_filtro['physicalexampath'].str.contains('gcs', case=False)].index)

gcs_temp = pd.merge(motor, eyes, on='patientunitstayid', how='outer')
gcs_temp2 = pd.merge(gcs_temp, verbal, on='patientunitstayid', how='outer')
gcs = pd.merge(gcs_temp2, gcs_total, on='patientunitstayid', how='outer')
gcs ['Total'] = gcs ['motor'] + gcs ['eyes'] + gcs ['verbal']

gcs['Total'] = gcs.apply(
    lambda row: row['gcs_total'] if not np.isnan(row['gcs_total']) and row['Total'] != row['gcs_total'] else row['Total'],
    axis=1
)
gcs = gcs.drop(['motor', 'eyes', 'verbal', 'gcs_total'], axis=1)
idx = gcs.groupby('patientunitstayid')['Total'].idxmin()
idx = idx.dropna()
gcs = gcs.loc[idx]

In [48]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

potassium = df_lab[df_lab['labname'].str.contains('potassium', case=False, na=False)]
potassium = potassium.query(querypred)
potassium = potassium[['patientunitstayid', 'labresult']].copy()

potassium['distancia'] = np.abs(potassium['labresult'] - 4.5)
potassium = potassium.dropna(subset=['distancia'])
potassium = potassium.loc[potassium.groupby('patientunitstayid')['distancia'].idxmax()]
potassium = potassium.reset_index(drop=True)
potassium = potassium.drop(['distancia'], axis=1)

potassium.rename(columns={'labresult' : 'potassium'}, inplace=True)

In [49]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

calcium = df_lab[df_lab['labname'].str.contains('calcium', case=False, na=False)]
calcium = calcium.query(querypred)
calcium = calcium[['patientunitstayid', 'labresult']].copy()

calcium['distancia'] = np.abs(calcium['labresult'] - 9.6)
calcium = calcium.dropna(subset=['distancia'])
calcium = calcium.loc[calcium.groupby('patientunitstayid')['distancia'].idxmax()]
calcium = calcium.reset_index(drop=True)
calcium = calcium.drop(['distancia'], axis=1)

calcium.rename(columns={'labresult' : 'calcium'}, inplace=True)

In [50]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

phosphate = df_lab[df_lab['labname'].str.contains('phosphate', case=False, na=False)]
phosphate = phosphate.query(querypred)
phosphate = phosphate[['patientunitstayid', 'labresult']].copy()

phosphate['distancia'] = np.abs(phosphate['labresult'] - 3.5)
phosphate = phosphate.dropna(subset=['distancia'])
phosphate = phosphate.loc[phosphate.groupby('patientunitstayid')['distancia'].idxmax()]
phosphate = phosphate.reset_index(drop=True)
phosphate = phosphate.drop(['distancia'], axis=1)

phosphate.rename(columns={'labresult' : 'phosphate'}, inplace=True)

In [51]:
tempo = "labresultoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

magnesium = df_lab[df_lab['labname'].str.contains('magnesium', case=False, na=False)]
magnesium = magnesium.query(querypred)
magnesium = magnesium[['patientunitstayid', 'labresult']].copy()
magnesium.rename(columns={'labresult' : 'magnesium'}, inplace=True)
magnesium = magnesium.groupby('patientunitstayid')['magnesium'].min().reset_index()

In [52]:
bioquimica = pd.merge(albumin, chloride, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, glucose, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, sodium, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, gcs, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, potassium, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, calcium, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, phosphate, on='patientunitstayid', how='outer')
bioquimica = pd.merge(bioquimica, magnesium, on='patientunitstayid', how='outer')
bioquimica

,patientunitstayid,albumin,chloride,glucose,sodium,Total,potassium,calcium,phosphate,magnesium
0,141168,3.1,101.0,95.0,139.0,15.0,4.0,8.8,NaN,NaN
1,141178,4.0,108.0,77.0,146.0,NaN,3.6,8.0,NaN,NaN
2,141179,NaN,109.0,54.0,146.0,NaN,3.6,7.8,NaN,1.8
3,141194,2.3,108.0,168.0,134.0,13.0,3.4,7.4,NaN,1.2
4,141196,2.5,97.0,144.0,135.0,NaN,4.1,9.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
195289,3353235,NaN,109.0,100.0,137.0,15.0,4.1,9.4,NaN,NaN
195290,3353237,3.5,101.0,139.0,140.0,14.0,3.6,9.0,NaN,NaN
195291,3353251,3.3,106.0,346.0,136.0,3.0,3.1,4.1,7.3,1.4
195292,3353254,3.9,117.0,181.0,141.0,15.0,5.7,6.4,3.7,1.7


# Medicamento


In [53]:
tempo = "drugorderoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

antibiotico = df_medication [['patientunitstayid', 'drugorderoffset', 'drugname', 'dosage']].copy()
antibiotico = antibiotico.query(querypred)

# Lista de antibióticos
antibioticos_list = [
    'Vancomycin', 'Metronidazole', 'Ceftriaxone', 'Cefazolin', 'Piperacillin-tazobactam',
    'Azithromycin', 'Cefepime', 'Penicillin', 'Clindamicina', 'Levofloxacino',
    'Meropenem', 'Levofloxacin', 'Ciprofloxacin', 'Zosyn'
]
regex_pattern = '|'.join(antibioticos_list)
antibiotico['drugname'] = antibiotico['drugname'].fillna('0')
# Criando a coluna 'antibiotico' com 1 se o paciente usou algum dos antibióticos e 0 caso contrário
antibiotico['antibiotico'] = antibiotico['drugname'].str.contains(regex_pattern, case=False, regex=True).astype(int)
antibiotico = antibiotico.loc[antibiotico.groupby('patientunitstayid')['antibiotico'].idxmax()]
antibiotico = antibiotico.reset_index(drop=True)
antibiotico = antibiotico.drop(['drugorderoffset', 'drugname' ,	'dosage'], axis=1)

# Cultura

In [54]:
tempo = "culturetakenoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

cultura = df_microlab [['patientunitstayid', 'culturetakenoffset', 'culturesite', 'organism']].copy()
cultura = cultura.query(querypred)

cultura = cultura.drop_duplicates(subset='patientunitstayid')
cultura['cultura'] = 1
cultura = cultura.drop(['culturetakenoffset', 'culturesite' ,	'organism'], axis=1)

# Cultura - bacterias específicas (será utilizado talvez em outro momento)

In [55]:
tempo = "culturetakenoffset"
querypred = f"{tempo} >= {t0} and {tempo} <= {t1}"

cultura_bac = df_microlab [['patientunitstayid', 'culturetakenoffset', 'culturesite', 'organism']].copy()
cultura_bac = cultura_bac.query(querypred)

# Lista de antibióticos
cultura_bac_list = [
    'gram negative rods', 'gram positive cocci', 'Acinetobacter baumanii', 'Staphylococcus aureus',
    'Clostridium difficile', 'Streptococcus pneumoniae', 'yeast',
    'Candida albicans', 'Klebsiella pneumoniae', 'Escherichia coli',
    'Pseudomonas aeruginosa', 'gram positive rods'
]
regex_pattern = '|'.join(cultura_bac_list)
cultura_bac['organism'] = cultura_bac['organism'].fillna('0')
# Criando a coluna 'cultura_bac' com 1 se o paciente usou algum dos antibióticos e 0 caso contrário
cultura_bac['cultura'] = cultura_bac['organism'].str.contains(regex_pattern, case=False, regex=True).astype(int)
cultura_bac = cultura_bac.loc[cultura_bac.groupby('patientunitstayid')['cultura'].idxmax()]
cultura_bac = cultura_bac.reset_index(drop=True)
cultura_bac = cultura_bac.drop(['culturetakenoffset', 'culturesite' ,	'organism'], axis=1)

# Merge set

In [56]:
# Internações: 0 por outro diagnóstico e 1 para Sepse
diagnostico = df_diagnosis [['patientunitstayid', 'diagnosisstring', 'diagnosisoffset']].copy()
diagnostico = diagnostico.query('diagnosisoffset > 1440 and diagnosisoffset < 4320')

# Criar a coluna de diagnóstico de sepse ou não
diagnostico['sepse_diagnostico'] = 0

# Verificar se ocorre a palavra "sepse" na coluna 'apacheadmissiondx' e atribuir 1 se ocorrer
diagnostico.loc[diagnostico['diagnosisstring'].str.contains('sepsis|septic', case=False, na=False), 'sepse_diagnostico'] = 1
diagnostico = diagnostico[['patientunitstayid','sepse_diagnostico']].copy()
diagnostico = diagnostico.groupby('patientunitstayid')['sepse_diagnostico'].max().reset_index()

In [57]:
# Internações: 0 por outro diagnóstico e 1 para Sepse
diagnostico_6 = df_diagnosis [['patientunitstayid', 'diagnosisstring', 'diagnosisoffset']].copy()
diagnostico_6 = diagnostico_6.query('diagnosisoffset <= 1440')

# Criar a coluna de diagnóstico de sepse ou não
diagnostico_6['sepse_diagnostico'] = 0

# Verificar se ocorre a palavra "sepse" na coluna 'apacheadmissiondx' e atribuir 1 se ocorrer
diagnostico_6.loc[diagnostico_6['diagnosisstring'].str.contains('sepsis', case=False, na=False), 'sepse_diagnostico'] = 1
diagnostico_6 = diagnostico_6[['patientunitstayid','sepse_diagnostico']].copy()
diagnostico_6 = diagnostico_6.groupby('patientunitstayid')['sepse_diagnostico'].max().reset_index()

diagnostico_6 = diagnostico_6.query('sepse_diagnostico == 1')

In [58]:
print ("Total de pessoas que entraram com diagnóstico Sepse: ", len (diagnostico_6))

Total de pessoas que entraram com diagnóstico Sepse:  21613


In [59]:
merged_df = pd.merge(diagnostico, diagnostico_6, on='patientunitstayid', how='left', indicator=True)
diagnostico = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge', 'sepse_diagnostico_y'])
diagnostico = diagnostico.rename(columns={'sepse_diagnostico_x' : 'sepse_diagnostico'})
diagnostico

,patientunitstayid,sepse_diagnostico
0,141515,1
1,141594,0
2,141822,0
3,141844,0
4,141866,0
...,...,...
47101,3353200,0
47102,3353201,0
47103,3353213,0
47104,3353226,0


In [60]:
len(diagnostico)

38135

### Diagnostico

In [61]:
diagnostico[diagnostico['sepse_diagnostico'] == 1]['sepse_diagnostico'].value_counts()

sepse_diagnostico
1    2696
Name: count, dtype: int64

In [62]:
# Criar a coluna sofa_diagnostico na tabela sofa
sofa['sofa_diagnostico'] = sofa['sofascore'].map(lambda x: 1 if x >= 2 else 0)
sofa2['sofa_diagnostico'] = sofa2['sofa2score'].map(lambda x: 1 if x >= 2 else 0)

# Agrupar por patientunitstayid e pegar o máximo de sofa_diagnostico
sofa = sofa.groupby('patientunitstayid')['sofa_diagnostico'].max().reset_index()
sofa2 = sofa2.groupby('patientunitstayid')['sofa_diagnostico'].max().reset_index()

# Merge cultura e antibioticos com os dois dataframes do SOFA
infeccao = pd.merge(cultura, antibiotico, on='patientunitstayid', how='outer')

sofa = pd.merge(sofa, infeccao, on='patientunitstayid', how='outer')
sofa2 = pd.merge(sofa2, infeccao, on='patientunitstayid', how='outer')

# Função para calcular SOFA + Suspeita de Infecção
def culturaesofa(row):
    if (row['cultura']==1 or row['antibiotico']==1) and row['sofa_diagnostico']==1:
        return 1
    else:
        return 0
    
sofa['sc_diagnostico'] = sofa.apply(culturaesofa, axis=1)
sofa2['sc_diagnostico2'] = sofa2.apply(culturaesofa, axis=1)

In [63]:
sofa

,patientunitstayid,sofa_diagnostico,cultura,antibiotico,sc_diagnostico
0,141168,1,NaN,0.0,0
1,141178,1,NaN,0.0,0
2,141179,1,NaN,0.0,0
3,141194,1,NaN,1.0,1
4,141196,1,NaN,1.0,1
...,...,...,...,...,...
200854,3353235,0,NaN,0.0,0
200855,3353237,0,NaN,0.0,0
200856,3353251,0,NaN,0.0,0
200857,3353254,0,NaN,0.0,0


In [64]:
sofa2

,patientunitstayid,sofa_diagnostico,cultura,antibiotico,sc_diagnostico2
0,141168,1,NaN,0.0,0
1,141178,0,NaN,0.0,0
2,141179,0,NaN,0.0,0
3,141194,0,NaN,1.0,0
4,141196,0,NaN,1.0,0
...,...,...,...,...,...
200854,3353235,0,NaN,0.0,0
200855,3353237,0,NaN,0.0,0
200856,3353251,0,NaN,0.0,0
200857,3353254,0,NaN,0.0,0


In [65]:
# Pacientes com sofa>=2 e suspeita de infecção nas primeiras 24 horas
sofa2 = sofa2.query('sc_diagnostico2 == 1')
len(sofa2)

5038

In [66]:
# Fazer a junção das tabelas e manter apenas os pacientes que não estão na tabela sofa2
mdf = pd.merge(sofa, sofa2, on='patientunitstayid', how='left', indicator=True)
sofa = mdf[mdf['_merge'] == 'left_only'].drop(columns=['_merge', 'sc_diagnostico2'])
sofa.rename(columns={'sc_diagnostico' : 'sofa_diagnostico'}, inplace=True)
sofa = sofa[['patientunitstayid', 'sofa_diagnostico']]

sofa

,patientunitstayid,sofa_diagnostico
0,141168,0
1,141178,0
2,141179,0
3,141194,1
4,141196,1
...,...,...
200854,3353235,0
200855,3353237,0
200856,3353251,0
200857,3353254,0


In [67]:
diagnostico = pd.merge(diagnostico, sofa, on='patientunitstayid', how='outer', indicator=True)
diagnostico = diagnostico.drop(['_merge'], axis=1)
diagnostico

,patientunitstayid,sepse_diagnostico,sofa_diagnostico
0,141168,NaN,0.0
1,141178,NaN,0.0
2,141179,NaN,0.0
3,141194,NaN,1.0
4,141196,NaN,1.0
...,...,...,...
196391,3353235,NaN,0.0
196392,3353237,NaN,0.0
196393,3353251,1.0,0.0
196394,3353254,NaN,0.0


### SOFA>=2 e Cultura + Diagnostico

In [68]:
diagnostico[diagnostico['sofa_diagnostico'] == 1]['sofa_diagnostico'].value_counts()

sofa_diagnostico
1.0    350
Name: count, dtype: int64

In [69]:
# Função para juntar os dois diagnósticos
def issepsis(row):
    if row['sepse_diagnostico']==1 or row['sofa_diagnostico']==1:
        return 1
    else:
        return 0

diagnostico['diagnostico'] = diagnostico.apply(issepsis, axis=1)
diagnostico = diagnostico.drop(['sepse_diagnostico', 'sofa_diagnostico'], axis=1)
diagnostico

,patientunitstayid,diagnostico
0,141168,0
1,141178,0
2,141179,0
3,141194,1
4,141196,1
...,...,...
196391,3353235,0
196392,3353237,0
196393,3353251,1
196394,3353254,0


In [70]:
df_final = pd.merge(characteristic, inflammation, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, pulmonary, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, cardiovascular, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, renal, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, hepatic, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, hematologic, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, bioquimica, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, antibiotico, on='patientunitstayid', how='outer')
df_final = pd.merge(df_final, diagnostico, on='patientunitstayid', how='inner')
df_final.head()

,patientunitstayid,gender,age,ethnicity,bands,CRP,ESR,temp,lactate,WBC,...,chloride,glucose,sodium,Total,potassium,calcium,phosphate,magnesium,antibiotico,diagnostico
0,141168,Female,70,Caucasian,NaN,NaN,NaN,NaN,NaN,9.8,...,101.0,95.0,139.0,15.0,4.0,8.8,NaN,NaN,0.0,0
1,141178,Female,52,Caucasian,NaN,NaN,NaN,NaN,NaN,7.6,...,108.0,77.0,146.0,NaN,3.6,8.0,NaN,NaN,0.0,0
2,141179,Female,52,Caucasian,NaN,NaN,NaN,NaN,NaN,NaN,...,109.0,54.0,146.0,NaN,3.6,7.8,NaN,1.8,0.0,0
3,141194,Male,68,Caucasian,NaN,NaN,NaN,NaN,1.9,14.1,...,108.0,168.0,134.0,13.0,3.4,7.4,NaN,1.2,1.0,1
4,141196,Male,71,Caucasian,NaN,NaN,NaN,NaN,0.8,26.2,...,97.0,144.0,135.0,NaN,4.1,9.0,NaN,NaN,1.0,1


# Statistics

In [71]:
pacientes = df_patient[df_patient['apacheadmissiondx'].str.contains('sepsis', case=False, na=False)]
len(pacientes)

23136

In [72]:
df_final[df_final['diagnostico'] == 1]['diagnostico'].value_counts()

diagnostico
1    3043
Name: count, dtype: int64

In [73]:
df_final[df_final['diagnostico'] == 0]['diagnostico'].value_counts()

diagnostico
0    193353
Name: count, dtype: int64

In [74]:
# Agrupar pacientes por 'uniquepid' e contar o número de internações para cada paciente
contagem_pacientes = df_patient['uniquepid'].value_counts().reset_index()

# Renomear colunas para clareza
contagem_pacientes.columns = ['uniquepid', 'frequencia']

# Filtrar apenas os pacientes que aparecem mais de uma vez
pacientes_repetidos = contagem_pacientes[contagem_pacientes['frequencia'] > 1]

# Número de pacientes repetidos
num_pacientes_repetidos = len(pacientes_repetidos)

# Exibir a quantidade de pacientes repetidos
print(f"Número de pacientes repetidos na base: {num_pacientes_repetidos}")

# Exibir a lista de pacientes repetidos (opcional)
print("Pacientes repetidos e suas frequências:")
print(pacientes_repetidos)


Número de pacientes repetidos na base: 38483
Pacientes repetidos e suas frequências:
        uniquepid  frequencia
0      021-239346          26
1      021-120471          26
2      006-100497          24
3       015-76326          24
4      006-181132          24
...           ...         ...
38478   035-20648           2
38479   035-16974           2
38480    035-6858           2
38481   002-76877           2
38482   002-47110           2

[38483 rows x 2 columns]


In [75]:
df_patient['uniquepid'].value_counts()

uniquepid
021-239346    26
021-120471    26
006-100497    24
015-76326     24
006-181132    24
              ..
035-20548      1
035-2622       1
035-12399      1
035-4075       1
002-9090       1
Name: count, Length: 139367, dtype: int64

# Numerical Values

In [76]:
df_final['age'] = df_final['age'].map(lambda x: '95' if x == '> 89' else x)
df_final['age'] = df_final['age'].astype(float)

In [77]:
df_final['gender'] = df_final['gender'].map(lambda x: 0 if x == 'Male' else 1)
df_final['gender'] = df_final['gender'].astype(int)

In [78]:
#Etnia 

ethnicity_one_hot = pd.get_dummies(df_final['ethnicity'], prefix='ethnicity')
df_final = pd.concat([df_final, ethnicity_one_hot], axis=1)
df_final.drop('ethnicity', axis=1, inplace=True)
df_final[ethnicity_one_hot.columns] = df_final[ethnicity_one_hot.columns].astype(int)

In [79]:
df_final.to_csv("df_final.csv", index=False)

## Limpar df_final

In [80]:
len(df_final)

196396

In [81]:
# Filtrar df_final removendo pacientes que permaneceram na UTI por menos de 24 horas
menos_24 = df_patient[df_patient['unitdischargeoffset'] < 1440]
df_final = df_final[~df_final['patientunitstayid'].isin(menos_24['patientunitstayid'])]

In [82]:
# Excluir pacientes da df_final que entraram pelo motivo de sepse
septics = df_patient[df_patient['apacheadmissiondx'].str.contains('sepsis|septic', case=False, na=False)]
df_final = df_final[~df_final['patientunitstayid'].isin(septics['patientunitstayid'])]

In [83]:
uti_total = df_patient [['patientunitstayid', 'hospitaladmitoffset']].copy()
uti_total = uti_total.query('hospitaladmitoffset < 0')
print ("Total de pessoas na UTI: ", len (uti_total))

Total de pessoas na UTI:  192665


In [84]:
len(df_final)

112479